In [1]:
from gensim.models import Word2Vec

cbow_model = Word2Vec.load('cbow_vec/word2vec_our.model')
mat_model = Word2Vec.load('mat2vec/training/models/pretrained_embeddings')
skip_model = Word2Vec.load('skip_vec/word2vec_our.model')

In [1]:
import json

with open('mat_dict.json', 'r', encoding='utf-8') as f:
    mat_dict = json.load(f)
print(len(mat_dict.keys()))   
etl_score = {}
with open('etl_score.txt', 'r', encoding='utf-8') as f:
    etl = f.readlines()
    for e in etl:
        es = e[:-1].split('\t')
        etl_score[es[0]] = es[1]
        
htl_score = {}
with open('htl_score.txt', 'r', encoding='utf-8') as f:
    htl = f.readlines()
    for h in htl:
        hs = h[:-1].split('\t')
        htl_score[hs[0]] = hs[1]  
        
with open('pair_list.json', 'r', encoding='utf-8') as f:
    pairs = json.load(f)
print(len(pairs))
# 4251 nomalization of abbrev
# arrange normalization by length (long first)
    
# based dictionary of materials
max_abbs = []
for p in pairs:
    max_abbs.append(p['max_abb'])

18218
4249


## word2vec evaluation

In [7]:
import csv

def model_eval(model, tl_):
    word_vectors = model.wv
    length = len(word_vectors.key_to_index)
    print(length)
    passivation_sims = word_vectors.most_similar('passivation', topn=length)
    contact_sims = word_vectors.most_similar('contact', topn=length)
    # contact_sims = word_vectors.most_similar('selectivity', topn=length)
    conductivity_sims = word_vectors.most_similar('conductivity', topn=length)
    passivation = word_vectors['passivation']
    contact = word_vectors['contact']
    # contact = word_vectors['selectivity']
    conductivity = word_vectors['conductivity']
    combine_vec = (passivation + contact) / 2
    combine_sims = word_vectors.most_similar(positive=[combine_vec], topn=length)
    combine_vec3 = (passivation + contact + conductivity) / 3
    combine_sims3 = word_vectors.most_similar(positive=[combine_vec3], topn=length)
    position = 0
    count = 0
    first_500 = 0
    tmp = 0
    test = []
    with open('skip_cbm3.csv', 'w', encoding='utf-8',newline='') as csvfile:
        f = csv.writer(csvfile)
        f.writerow(['rank','name','freq_N', 'freq_Y'])
        for p_set in combine_sims3:
            p = p_set[0]
            if p in mat_dict.keys():
                if 'Y' in mat_dict[p].keys():
                    y = mat_dict[p]['Y']
                else:
                    y = 0
                if 'N' in mat_dict[p].keys():
                    n = mat_dict[p]['N']
                else:
                    n = 0
                if n+y>3:
                    tmp += 1            
                    f.writerow([str(tmp),p,str(n), str(y)])
            if p in list(tl_.keys())[:30]:
                test.append(p)
                position += tmp
                count += 1
                if tmp < 500:
                    first_500 += 1
    for t in list(tl_.keys())[:30]:
        if t not in test:
            print(t)
    print(position, count)
    return position/count, first_500            

In [8]:
print(model_eval(skip_model, htl_score))

112787
123654 30
(4121.8, 0)


## BERT Evaluation

In [4]:
import csv

# with open('MatSciBERT/sorted_score3.json', 'r', encoding='utf-8') as f:
# mat_bert/sorted_score_sen.json
with open('checkpoint-280000/sorted_score_our.json', 'r', encoding='utf-8') as f:
    bert_sims = json.load(f)
position = 0
count = 0
first_500 = 0
tmp = 0
test = []
tl_ = etl_score
with open('bert_sen_ber_cbm.csv', 'w', encoding='utf-8',newline='') as csvfile:
    f = csv.writer(csvfile)
    f.writerow(['rank','name','freq_N', 'freq_Y'])
    for b_set in bert_sims:
        b = b_set[0]
        if b in mat_dict.keys():
            if 'Y' in mat_dict[b].keys():
                y = mat_dict[b]['Y']
            else:
                y = 0
            if 'N' in mat_dict[b].keys():
                n = mat_dict[b]['N']
            else:
                n = 0
            if n+y>3:
                tmp += 1            
                f.writerow([str(tmp),b,str(n), str(y)])
            if b in list(tl_.keys())[:30]:
                test.append(b)
                position += tmp
                count += 1
                if tmp < 500:
                    first_500 += 1
for t in list(tl_.keys())[:30]:
    if t not in test:
        print(t)
print(position, count)
print(position/count, first_500) 

81517 30
2717.233333333333 7


In [ ]:
# cbow combine_sims3 etl, 28248 30 941.6, 17
# cbow combine_sims3 htl, 34515 30 1150.5, 13
# cbow combine_sims etl, 29356 30 978.53, 16
# cbow combine_sims htl, 35232 30 1174.4, 14

# skip combine_sims3 etl, 88311 30 2943.7, 0
# skip combine_sims3 htl, 123654 30 4121.8, 0
# skip combine_sims etl, 87639 30 2921.3, 2 
# skip combine_sims htl, 126369 30 4212.3, 1

# mat combine_sims3 etl, 52933 30 1764.43, 3
# mat combine_sims3 htl, 58482 27 2166.0, 3
# mat combine_sims etl, 58668 30 1955.6, 4 
# mat combine_sims htl, 55769 27 2065.52, 2

# bert_d_ind combine_sims3 etl,90383 30 3012.77 4
# bert_d_ind combine_sims3 htl,78509 30 2616.97 7
# bert_d_ind combine_sims etl,92371 30 3079.03 2 
# bert_d_ind combine_sims htl,82341 30 2744.7 5

# bert_d_ber combine_sims3 etl,79874 30 2662.47 3
# bert_d_ber combine_sims3 htl,55485 30 1849.5 4
# bert_d_ber combine_sims etl,79874 30 2662.47 3 
# bert_d_ber combine_sims htl,55485 30 1849.5 4

# bert_d_our 


# ber
# v_test = get_embedding2('contactpassivationconductivity', 'contact passivation conductivity', tokenizer, model)
# bert_a combine_sims3 etl,54017 30 1800.56 8
# bert_a combine_sims3 htl,56665 30 1888.83 6

# v_test = get_embedding2('contactpassivation', 'contact passivation', tokenizer, model)
# bert_a combine_sims etl, 42096 30 1403.2 9
# bert_a combine_sims htl,61032 30 2034.4 5

# v_test = get_embedding2('contactpassivation', 'contact passivation conductivity', tokenizer, model)
# bert_a combine_sims etl, 43382 30 1446.07 11
# bert_a combine_sims htl,58780 30 1959.33 4

# v1 = get_embedding1('passivation', tokenizer, model)
# v = torch.mean(torch.stack([v1, v2, v3]), 0)
# bert_a combine_sims etl,43678 30 1455.93 7
# bert_a combine_sims htl,67068 30 2235.6 4

# sen
# sen = 'passivation, conductivity, and selectivity are often acknowledged as the three requirements for optimal ' \
      # 'contacts to photovoltaic solar cells '
# v1 = get_embedding2('conductivity', sen, tokenizer, model)
# v2 = get_embedding2('passivation', sen, tokenizer, model)
# v3 = get_embedding2('selectivity', sen, tokenizer, model)
# bert_a_ber combine_sims etl,48623 30 1620.76 11
# bert_a_ber combine_sims htl,45402 30 1513.4 9
# bert_a_our combine_sims etl,
# bert_a_our combine_sims htl,